In [1]:
import joblib
import xarray as xr
import pandas as pd
import anndata
import numpy as np
import seaborn as sns
import anndata
from scipy.sparse import csr_matrix, vstack
import matplotlib.pyplot as plt

from concurrent.futures import as_completed, ProcessPoolExecutor

## Load Data

### Cell type phylogeny

In [2]:
node_cluster_dict = joblib.load(
    '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial-DissectionRegion/DMG/ITSpatial.pvclust.non_singleton_node_dict.lib'
)

cluster_dendro = joblib.load(
    '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial-DissectionRegion/DMG/ITSpatial.pvclust.dendrogram.lib'
)
cluster_linkage = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial-DissectionRegion/DMG/ITSpatial.pvclust.linkage.csv', 
    index_col=0).values
cluster_linkage_order = pd.read_csv(
    '/home/hanliu/project/mouse_rostral_brain/study/ITSpatial-DissectionRegion/DMG/ITSpatial.pvclust.linkage.orders.txt', 
    index_col=0, header=None
).index

### DMR

#### DMR Rate

In [3]:
dmr_rate = pd.read_hdf(
    '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_DissectionRegion/DMRInfo.h5',
    key='Rate').loc[:, cluster_dendro['ivl']].copy()

#### DMR Hits

In [4]:
hypo_hits = anndata.read_h5ad(
    '/home/hanliu/project/mouse_rostral_brain/DMR/ITSpatial_DissectionRegion/HypoDMR/TotalHits.h5ad'
)[:, cluster_dendro['ivl']].copy()

### Make pairs

In [5]:
from itertools import combinations

In [6]:
cluster_pairs = list(combinations(cluster_dendro['ivl'], 2))
print(len(cluster_pairs), 'pairwise comparison to test.')

2628 pairwise comparison to test.


## Select DMR Per Pair

In [8]:
dmr_rate_delta = 0.3

In [9]:
pairs = []
records = []

for i, (a, b) in enumerate(cluster_pairs):
    rate_delta = dmr_rate[a] - dmr_rate[b]
    # only one of the cluster is 1, means one hypo, the other not
    hypo_judge = (hypo_hits.obs_vector(a) + hypo_hits.obs_vector(b)) == 1
    rate_delta = rate_delta[hypo_judge].copy()
    # top 100k (or less) most diff DMRs
    a_hypo = rate_delta[rate_delta < -dmr_rate_delta].sort_values()[:100000]
    b_hypo = rate_delta[rate_delta > dmr_rate_delta].sort_values(ascending=False)[:100000]
    
    sparse_data = csr_matrix(dmr_rate.index.isin(a_hypo.index | b_hypo.index)[None, :])

    pairs.append([a, b])
    records.append(sparse_data)
    print(i, a, a_hypo.size, f'{a_hypo.iat[-1]:.2f}', b, b_hypo.size, f'{b_hypo.iat[-1]:.2f}', sep='\t')

0	IT-L4+MOp-2	368	-0.30	IT-L4+SSs-1	700	0.30
1	IT-L4+MOp-2	295	-0.30	IT-L4+SSs-2	1168	0.30
2	IT-L4+MOp-2	2557	-0.30	IT-L4+SSp-5	1042	0.30
3	IT-L4+MOp-2	380	-0.30	IT-L4+SSp-1	730	0.30
4	IT-L4+MOp-2	836	-0.30	IT-L4+SSp-2	2075	0.30
5	IT-L4+MOp-2	1072	-0.30	IT-L4+SSp-3	2454	0.30
6	IT-L4+MOp-2	1523	-0.30	IT-L4+SSp-4	1250	0.30
7	IT-L4+MOp-2	4661	-0.30	IT-L4+MOs-3	2445	0.30
8	IT-L4+MOp-2	951	-0.30	IT-L4+MOp-3	199	0.30
9	IT-L4+MOp-2	4027	-0.30	IT-L4+MOp-4	660	0.30
10	IT-L4+MOp-2	1616	-0.30	IT-L4+MOp-1	2216	0.30
11	IT-L4+MOp-2	5225	-0.30	IT-L4+MOs-1	6299	0.30
12	IT-L4+MOp-2	17378	-0.30	IT-L23+SSp-1	24923	0.30
13	IT-L4+MOp-2	18081	-0.30	IT-L23+SSs-1	19129	0.30
14	IT-L4+MOp-2	17402	-0.30	IT-L23+SSs-2	31391	0.30
15	IT-L4+MOp-2	15873	-0.30	IT-L23+SSp-2	27507	0.30
16	IT-L4+MOp-2	19606	-0.30	IT-L23+SSp-4	19344	0.30
17	IT-L4+MOp-2	17392	-0.30	IT-L23+SSp-3	23618	0.30
18	IT-L4+MOp-2	21090	-0.30	IT-L23+SSp-5	14619	0.30
19	IT-L4+MOp-2	21494	-0.30	IT-L23+MOp-4	17420	0.30
20	IT-L4+MOp-2	18672	-0.30	IT-L23+M

166	IT-L4+SSs-2	32124	-0.30	IT-L23+MOs-2	19016	0.30
167	IT-L4+SSs-2	32440	-0.30	IT-L23+MOs-3	16644	0.30
168	IT-L4+SSs-2	30070	-0.30	IT-L23+ORB	4646	0.30
169	IT-L4+SSs-2	33754	-0.30	IT-L23+ACA-2	11581	0.30
170	IT-L4+SSs-2	32947	-0.30	IT-L23+ACA-3	9600	0.30
171	IT-L4+SSs-2	34475	-0.30	IT-L23+ACA-1	12601	0.30
172	IT-L4+SSs-2	35976	-0.30	IT-L23+PFC-1	17790	0.30
173	IT-L4+SSs-2	35322	-0.30	IT-L23+PFC-2	15717	0.30
174	IT-L4+SSs-2	34602	-0.30	IT-L6+SSp-1	6598	0.30
175	IT-L4+SSs-2	35570	-0.30	IT-L6+SSs-1	3250	0.30
176	IT-L4+SSs-2	34925	-0.30	IT-L6+SSs-2	4918	0.30
177	IT-L4+SSs-2	33928	-0.30	IT-L6+SSp-2	9858	0.30
178	IT-L4+SSs-2	34597	-0.30	IT-L6+SSp-4	4825	0.30
179	IT-L4+SSs-2	34138	-0.30	IT-L6+SSp-3	7191	0.30
180	IT-L4+SSs-2	35324	-0.30	IT-L6+SSp-5	4306	0.30
181	IT-L4+SSs-2	36416	-0.30	IT-L6+ACA-2	7403	0.30
182	IT-L4+SSs-2	36283	-0.30	IT-L6+ACA-3	8620	0.30
183	IT-L4+SSs-2	36554	-0.30	IT-L6+ACA-1	8363	0.30
184	IT-L4+SSs-2	36909	-0.30	IT-L6+PFC-1	7167	0.30
185	IT-L4+SSs-2	37027	-0.30	IT-L6+PFC-

329	IT-L4+SSp-1	25896	-0.30	IT-L5+MOs-2	10506	0.30
330	IT-L4+SSp-1	24945	-0.30	IT-L5+MOs-3	7050	0.30
331	IT-L4+SSp-1	19305	-0.30	IT-L5+MOp-2	13150	0.30
332	IT-L4+SSp-1	20437	-0.30	IT-L5+MOp-3	10124	0.30
333	IT-L4+SSp-1	24194	-0.30	IT-L5+MOp-4	7222	0.30
334	IT-L4+SSp-1	16131	-0.30	IT-L5+SSp-4	5859	0.30
335	IT-L4+SSp-1	20742	-0.30	IT-L5+SSp-5	4572	0.30
336	IT-L4+SSp-1	17643	-0.30	IT-L5+SSs-1	10031	0.30
337	IT-L4+SSp-1	13610	-0.30	IT-L5+SSp-2	13256	0.30
338	IT-L4+SSp-1	14014	-0.30	IT-L5+SSp-3	8555	0.30
339	IT-L4+SSp-1	15759	-0.30	IT-L5+SSp-1	8090	0.30
340	IT-L4+SSp-1	14331	-0.30	IT-L5+SSs-2	12574	0.30
341	IT-L4+SSp-1	28754	-0.30	IT-L5+ACA-1	6791	0.30
342	IT-L4+SSp-1	27957	-0.30	IT-L5+ACA-2	8796	0.30
343	IT-L4+SSp-1	25803	-0.30	IT-L5+ACA-3	9390	0.30
344	IT-L4+SSp-1	30822	-0.30	IT-L5+PFC-1	12926	0.30
345	IT-L4+SSp-1	30684	-0.30	IT-L5+PFC-2	8758	0.30
346	IT-L4+SSp-1	22295	-0.30	IT-L5+MOp-1	14449	0.30
347	IT-L4+SSp-1	25102	-0.30	IT-L5+MOs-1	18172	0.30
348	IT-L4+SSp-1	29098	-0.30	IT-L5+AI	1836

492	IT-L4+SSp-4	15032	-0.30	IT-L23+SSp-4	18196	0.30
493	IT-L4+SSp-4	13574	-0.30	IT-L23+SSp-3	22812	0.30
494	IT-L4+SSp-4	16014	-0.30	IT-L23+SSp-5	13107	0.30
495	IT-L4+SSp-4	17625	-0.30	IT-L23+MOp-4	17249	0.30
496	IT-L4+SSp-4	16174	-0.30	IT-L23+MOp-2	27140	0.30
497	IT-L4+SSp-4	16529	-0.30	IT-L23+MOp-3	23023	0.30
498	IT-L4+SSp-4	21237	-0.30	IT-L23+AI	16765	0.30
499	IT-L4+SSp-4	18606	-0.30	IT-L23+MOp-1	30542	0.30
500	IT-L4+SSp-4	19744	-0.30	IT-L23+MOs-1	23074	0.30
501	IT-L4+SSp-4	18783	-0.30	IT-L23+MOs-2	19111	0.30
502	IT-L4+SSp-4	18646	-0.30	IT-L23+MOs-3	16598	0.30
503	IT-L4+SSp-4	18315	-0.30	IT-L23+ORB	5042	0.30
504	IT-L4+SSp-4	18949	-0.30	IT-L23+ACA-2	10828	0.30
505	IT-L4+SSp-4	18044	-0.30	IT-L23+ACA-3	8734	0.30
506	IT-L4+SSp-4	19582	-0.30	IT-L23+ACA-1	11890	0.30
507	IT-L4+SSp-4	21206	-0.30	IT-L23+PFC-1	17576	0.30
508	IT-L4+SSp-4	20341	-0.30	IT-L23+PFC-2	15137	0.30
509	IT-L4+SSp-4	20393	-0.30	IT-L6+SSp-1	6437	0.30
510	IT-L4+SSp-4	21225	-0.30	IT-L6+SSs-1	3230	0.30
511	IT-L4+SSp-4	20750	-

655	IT-L4+MOp-3	10390	-0.30	IT-L5+MOs-3	6374	0.30
656	IT-L4+MOp-3	9129	-0.30	IT-L5+MOp-2	13494	0.30
657	IT-L4+MOp-3	9296	-0.30	IT-L5+MOp-3	10085	0.30
658	IT-L4+MOp-3	10534	-0.30	IT-L5+MOp-4	6636	0.30
659	IT-L4+MOp-3	8364	-0.30	IT-L5+SSp-4	6933	0.30
660	IT-L4+MOp-3	9837	-0.30	IT-L5+SSp-5	4669	0.30
661	IT-L4+MOp-3	9690	-0.30	IT-L5+SSs-1	12135	0.30
662	IT-L4+MOp-3	8118	-0.30	IT-L5+SSp-2	17269	0.30
663	IT-L4+MOp-3	7891	-0.30	IT-L5+SSp-3	10713	0.30
664	IT-L4+MOp-3	8787	-0.30	IT-L5+SSp-1	10045	0.30
665	IT-L4+MOp-3	8461	-0.30	IT-L5+SSs-2	16353	0.30
666	IT-L4+MOp-3	13039	-0.30	IT-L5+ACA-1	6251	0.30
667	IT-L4+MOp-3	12955	-0.30	IT-L5+ACA-2	8511	0.30
668	IT-L4+MOp-3	12245	-0.30	IT-L5+ACA-3	9868	0.30
669	IT-L4+MOp-3	14464	-0.30	IT-L5+PFC-1	12292	0.30
670	IT-L4+MOp-3	14325	-0.30	IT-L5+PFC-2	8137	0.30
671	IT-L4+MOp-3	8831	-0.30	IT-L5+MOp-1	13606	0.30
672	IT-L4+MOp-3	9349	-0.30	IT-L5+MOs-1	16155	0.30
673	IT-L4+MOp-3	13829	-0.30	IT-L5+AI	18272	0.30
674	IT-L4+MOp-3	11390	-0.30	IT-L5+ORB	8606	0.30
675	I

...

1938	IT-L6+SSs-1	280	-0.30	IT-L6+MOp-2	3708	0.30
1939	IT-L6+SSs-1	256	-0.30	IT-L6+MOp-3	4648	0.30
1940	IT-L6+SSs-1	462	-0.30	IT-L6+MOp-4	3332	0.30
1941	IT-L6+SSs-1	1732	-0.30	IT-L5+MOs-2	8833	0.30
1942	IT-L6+SSs-1	2456	-0.30	IT-L5+MOs-3	7247	0.30
1943	IT-L6+SSs-1	1753	-0.30	IT-L5+MOp-2	14937	0.30
1944	IT-L6+SSs-1	2064	-0.30	IT-L5+MOp-3	12080	0.30
1945	IT-L6+SSs-1	2022	-0.30	IT-L5+MOp-4	7070	0.30
1946	IT-L6+SSs-1	2594	-0.30	IT-L5+SSp-4	11326	0.30
1947	IT-L6+SSs-1	2610	-0.30	IT-L5+SSp-5	6836	0.30
1948	IT-L6+SSs-1	2276	-0.30	IT-L5+SSs-1	14924	0.30
1949	IT-L6+SSs-1	2492	-0.30	IT-L5+SSp-2	24161	0.30
1950	IT-L6+SSs-1	2501	-0.30	IT-L5+SSp-3	17148	0.30
1951	IT-L6+SSs-1	2151	-0.30	IT-L5+SSp-1	13619	0.30
1952	IT-L6+SSs-1	2512	-0.30	IT-L5+SSs-2	23132	0.30
1953	IT-L6+SSs-1	3026	-0.30	IT-L5+ACA-1	7265	0.30
1954	IT-L6+SSs-1	3128	-0.30	IT-L5+ACA-2	10376	0.30
1955	IT-L6+SSs-1	3344	-0.30	IT-L5+ACA-3	13204	0.30
1956	IT-L6+SSs-1	2329	-0.30	IT-L5+PFC-1	10612	0.30
1957	IT-L6+SSs-1	2597	-0.30	IT-L5+PFC-2	75

2102	IT-L6+SSp-5	1457	-0.30	IT-L6+ACA-1	3445	0.30
2103	IT-L6+SSp-5	2181	-0.30	IT-L6+PFC-1	3511	0.30
2104	IT-L6+SSp-5	1893	-0.30	IT-L6+PFC-2	2902	0.30
2105	IT-L6+SSp-5	1524	-0.30	IT-L6+AI	3192	0.30
2106	IT-L6+SSp-5	637	-0.30	IT-L6+MOs-2	8082	0.30
2107	IT-L6+SSp-5	421	-0.30	IT-L6+MOs-3	5893	0.30
2108	IT-L6+SSp-5	257	-0.30	IT-L6+MOp-2	3547	0.30
2109	IT-L6+SSp-5	133	-0.30	IT-L6+MOp-3	3350	0.30
2110	IT-L6+SSp-5	207	-0.30	IT-L6+MOp-4	1845	0.30
2111	IT-L6+SSp-5	3148	-0.30	IT-L5+MOs-2	10070	0.30
2112	IT-L6+SSp-5	3652	-0.30	IT-L5+MOs-3	7669	0.30
2113	IT-L6+SSp-5	2807	-0.30	IT-L5+MOp-2	15704	0.30
2114	IT-L6+SSp-5	2917	-0.30	IT-L5+MOp-3	12219	0.30
2115	IT-L6+SSp-5	2706	-0.30	IT-L5+MOp-4	6976	0.30
2116	IT-L6+SSp-5	3553	-0.30	IT-L5+SSp-4	11274	0.30
2117	IT-L6+SSp-5	3567	-0.30	IT-L5+SSp-5	6695	0.30
2118	IT-L6+SSp-5	3920	-0.30	IT-L5+SSs-1	16075	0.30
2119	IT-L6+SSp-5	3485	-0.30	IT-L5+SSp-2	24221	0.30
2120	IT-L6+SSp-5	3350	-0.30	IT-L5+SSp-3	17076	0.30
2121	IT-L6+SSp-5	3290	-0.30	IT-L5+SSp-1	14098	0.30


2266	IT-L6+PFC-2	5678	-0.30	IT-L5+SSp-1	15058	0.30
2267	IT-L6+PFC-2	6067	-0.30	IT-L5+SSs-2	24974	0.30
2268	IT-L6+PFC-2	5051	-0.30	IT-L5+ACA-1	6301	0.30
2269	IT-L6+PFC-2	5544	-0.30	IT-L5+ACA-2	9648	0.30
2270	IT-L6+PFC-2	6193	-0.30	IT-L5+ACA-3	12892	0.30
2271	IT-L6+PFC-2	3293	-0.30	IT-L5+PFC-1	8177	0.30
2272	IT-L6+PFC-2	3797	-0.30	IT-L5+PFC-2	5770	0.30
2273	IT-L6+PFC-2	4126	-0.30	IT-L5+MOp-1	15285	0.30
2274	IT-L6+PFC-2	4164	-0.30	IT-L5+MOs-1	17876	0.30
2275	IT-L6+PFC-2	3519	-0.30	IT-L5+AI	14393	0.30
2276	IT-L6+PFC-2	5177	-0.30	IT-L5+ORB	9364	0.30
2277	IT-L6+AI	1390	-0.30	IT-L6+MOs-2	7111	0.30
2278	IT-L6+AI	1470	-0.30	IT-L6+MOs-3	5820	0.30
2279	IT-L6+AI	1229	-0.30	IT-L6+MOp-2	4567	0.30
2280	IT-L6+AI	1295	-0.30	IT-L6+MOp-3	6111	0.30
2281	IT-L6+AI	2014	-0.30	IT-L6+MOp-4	4353	0.30
2282	IT-L6+AI	3789	-0.30	IT-L5+MOs-2	7984	0.30
2283	IT-L6+AI	5267	-0.30	IT-L5+MOs-3	6908	0.30
2284	IT-L6+AI	4530	-0.30	IT-L5+MOp-2	14993	0.30
2285	IT-L6+AI	5248	-0.30	IT-L5+MOp-3	12269	0.30
2286	IT-L6+AI	5170	-0.30

2430	IT-L5+MOs-2	4515	-0.30	IT-L5+ACA-2	3512	0.30
2431	IT-L5+MOs-2	7473	-0.30	IT-L5+ACA-3	7442	0.30
2432	IT-L5+MOs-2	2665	-0.30	IT-L5+PFC-1	1859	0.30
2433	IT-L5+MOs-2	2607	-0.30	IT-L5+PFC-2	996	0.30
2434	IT-L5+MOs-2	185	-0.30	IT-L5+MOp-1	687	0.30
2435	IT-L5+MOs-2	191	-0.30	IT-L5+MOs-1	903	0.30
2436	IT-L5+MOs-2	1035	-0.30	IT-L5+AI	2881	0.30
2437	IT-L5+MOs-2	1807	-0.30	IT-L5+ORB	1267	0.30
2438	IT-L5+MOs-3	291	-0.30	IT-L5+MOp-2	1890	0.30
2439	IT-L5+MOs-3	225	-0.30	IT-L5+MOp-3	792	0.30
2440	IT-L5+MOs-3	377	-0.30	IT-L5+MOp-4	453	0.30
2441	IT-L5+MOs-3	1375	-0.30	IT-L5+SSp-4	2694	0.30
2442	IT-L5+MOs-3	1358	-0.30	IT-L5+SSp-5	921	0.30
2443	IT-L5+MOs-3	1556	-0.30	IT-L5+SSs-1	5592	0.30
2444	IT-L5+MOs-3	1581	-0.30	IT-L5+SSp-2	11452	0.30
2445	IT-L5+MOs-3	1192	-0.30	IT-L5+SSp-3	4806	0.30
2446	IT-L5+MOs-3	1185	-0.30	IT-L5+SSp-1	4093	0.30
2447	IT-L5+MOs-3	1444	-0.30	IT-L5+SSs-2	9418	0.30
2448	IT-L5+MOs-3	1850	-0.30	IT-L5+ACA-1	1277	0.30
2449	IT-L5+MOs-3	1638	-0.30	IT-L5+ACA-2	1907	0.30
2450	IT-L5+MOs-

2596	IT-L5+ACA-1	2632	-0.30	IT-L5+MOp-1	9403	0.30
2597	IT-L5+ACA-1	1823	-0.30	IT-L5+MOs-1	9037	0.30
2598	IT-L5+ACA-1	2769	-0.30	IT-L5+AI	10360	0.30
2599	IT-L5+ACA-1	1509	-0.30	IT-L5+ORB	3103	0.30
2600	IT-L5+ACA-2	350	-0.30	IT-L5+ACA-3	496	0.30
2601	IT-L5+ACA-2	3968	-0.30	IT-L5+PFC-1	4424	0.30
2602	IT-L5+ACA-2	1925	-0.30	IT-L5+PFC-2	1045	0.30
2603	IT-L5+ACA-2	4640	-0.30	IT-L5+MOp-1	10240	0.30
2604	IT-L5+ACA-2	4035	-0.30	IT-L5+MOs-1	10916	0.30
2605	IT-L5+ACA-2	5018	-0.30	IT-L5+AI	11296	0.30
2606	IT-L5+ACA-2	2789	-0.30	IT-L5+ORB	3250	0.30
2607	IT-L5+ACA-3	9002	-0.30	IT-L5+PFC-1	8614	0.30
2608	IT-L5+ACA-3	6679	-0.30	IT-L5+PFC-2	3763	0.30
2609	IT-L5+ACA-3	8161	-0.30	IT-L5+MOp-1	13482	0.30
2610	IT-L5+ACA-3	8334	-0.30	IT-L5+MOs-1	15838	0.30
2611	IT-L5+ACA-3	9213	-0.30	IT-L5+AI	15210	0.30
2612	IT-L5+ACA-3	6324	-0.30	IT-L5+ORB	5409	0.30
2613	IT-L5+PFC-1	343	-0.30	IT-L5+PFC-2	140	0.30
2614	IT-L5+PFC-1	2543	-0.30	IT-L5+MOp-1	6728	0.30
2615	IT-L5+PFC-1	795	-0.30	IT-L5+MOs-1	4262	0.30
2616	IT-L5+PF

In [10]:
adata = anndata.AnnData(X=vstack(records),
                        obs=pd.DataFrame(pairs, columns=['cluster_a', 'cluster_b']),
                        var=pd.DataFrame([], index=dmr_rate.index))

Transforming to str index.


In [11]:
adata.write_h5ad('PairwiseDMR.h5ad')

... storing 'cluster_a' as categorical
... storing 'cluster_b' as categorical
